In [4]:
# def interpret_circular_shift(present_D_mean, future_D_mean):
#     # Calculate the circular difference
#     circular_diff = ((future_D_mean - present_D_mean + 182.625) % 365.25) - 182.625
    
#     # Define end-of-year and start-of-year thresholds
#     end_of_year_threshold = 330
#     start_of_year_threshold = 30
    
#     # Detect if we are straddling the December-January boundary
#     if present_D_mean >= end_of_year_threshold and future_D_mean <= start_of_year_threshold:
#         # We’re crossing from December to January
#         interpretation = f"The negative circular difference ({circular_diff:.2f} days) crosses December-January, meaning events are shifting later into the winter season."
#     elif circular_diff > 0:
#         interpretation = f"Events are happening {circular_diff:.2f} days later in the future."
#     else:
#         interpretation = f"Events are happening {-circular_diff:.2f} days earlier in the future."

#     return circular_diff, interpretation

# # Example usage
# present_D_mean = 350  # Late December
# future_D_mean = 15    # Early January

# circular_diff, interpretation = interpret_circular_shift(present_D_mean, future_D_mean)
# print(interpretation)

In [5]:
import pickle
import pandas as pd
import numpy as np
import datetime
import json

pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [6]:
from FormatEvents_Functions import *

In [7]:
home_dir = '/nfs/a319/gy17m2a/PhD/'
home_dir2 = '/nfs/a161/gy17m2a/PhD/'

In [8]:
durations = ['0.5', '1', '2', '3', '6', '12', '24']

### UKCP18 data

### Join together lists for different ensemble members

In [9]:
events_props_dict_present = []
ems_present = ['bc005', 'bc006', 'bc007', 'bc009', 'bc010', 'bc011', 'bc012', 'bc013', 'bc015', 'bc016', 'bc017', 'bc018']
for em in ems_present:
    with open(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/Present/event_props_dict_{em}.pickle", 'rb') as handle:
        one_events_props_dict_present = pickle.load(handle)    
    events_props_dict_present = events_props_dict_present + one_events_props_dict_present
    
## Join into one dataframe    
present = pd.DataFrame(events_props_dict_present)
present['Climate'] = 'Present'    

In [10]:
events_props_dict_future = []
# ems_future = ['bb195', 'bb192', 'bb198', 'bb208', 'bb225','bb222', 'bb201', 'bb204', 'bb216', 'bb219', 'bb211']
ems_future = ['bb195', 'bb198', 'bb192', 'bb208', 'bb225','bb222', 'bb201', 'bb204', 'bb216', 'bb219', 'bb211', 'bb189'] #bb195, #bb198
for em in ems_future:
    with open(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/event_props_dict_{em}.pickle", 'rb') as handle:
        one_events_props_dict_future = pickle.load(handle)    
    events_props_dict_future = events_props_dict_future + one_events_props_dict_future
    
## Join into one dataframe
future = pd.DataFrame(events_props_dict_future)
future['Climate'] = 'Future'

## Make a check on number of files (could shift this to the checking script)
NB - the method of searching on part1 doesnt work, because the filename only represents on of the files that is represented by that event


24529 is 19 * 1291 and is the number we expert with no part1s for one ensemble member.  
For 12 ems it becomes 24529 * 12 = 294348

### Create one dataframe containing both present and future

In [11]:
df_long = pd.concat([present, future])

# Add D variable (day of year) and date
df_long['D'] = (df_long['theta'] * 365.25) / (2 * np.pi)
df_long['date'] = df_long.apply(lambda row: date_from_D(row['D'], row['year']), axis=1)
df_long['season'] = df_long['date'].apply(get_season)

### Check the number of files for each duration
NB: Number of files for 24h duration is longer due to compound events  
Checked this by filtering only rows with part0

### Remove entries which are less than 1.5 hours

In [12]:
df_long_with_short_durations_kept = df_long.copy()

In [13]:
df_long = df_long[df_long['duration'] >=1.5]
present = present[present['duration'] >=1.5]
future = future[future['duration'] >=1.5]
# nan_rows = df_long[df_long['D50'].isna()]

### NIMROD data

In [10]:
with open(home_dir +  f"ProcessedData/AMAX_Events/NIMROD_30mins/event_props_dict.pickle", 'rb') as handle:
    events_props_dict_nimrod = pickle.load(handle)

In [11]:
nimrod = pd.DataFrame(events_props_dict_nimrod)
# Add D variable (day of year) and date
nimrod['D'] = (nimrod['theta'] * 365.25) / (2 * np.pi)
nimrod['date'] = nimrod.apply(lambda row: date_from_D(row['D'], row['year']), axis=1)
nimrod['season'] = nimrod['date'].apply(get_season)

In [12]:
# Ensure that values are treated as lists. If any single numbers are not in a list, convert them to lists.
nimrod['dur_for_which_this_is_amax'] = nimrod['dur_for_which_this_is_amax'].apply(lambda x: x if isinstance(x, list) else [x])

# Flatten the lists and count occurrences of each number
all_numbers = [num for sublist in nimrod['dur_for_which_this_is_amax'] for num in sublist]
number_counts = pd.Series(all_numbers).value_counts()

# Show the result
print(number_counts)

24     19462
0.5    19363
12     14560
6      11379
2       8136
1       7536
3       6830
dtype: int64


In [13]:
# Ensure that values are treated as lists. If any single numbers are not in a list, convert them to lists.
nimrod['dur_for_which_this_is_amax'] = nimrod['dur_for_which_this_is_amax'].apply(lambda x: x if isinstance(x, list) else [x])

# Flatten the lists and count occurrences of each number
all_numbers = [num for sublist in nimrod['dur_for_which_this_is_amax'] for num in sublist]
number_counts = pd.Series(all_numbers).value_counts()

# Show the result
print(number_counts)

24     19462
0.5    19363
12     14560
6      11379
2       8136
1       7536
3       6830
dtype: int64


In [17]:
nimrod.to_csv(home_dir + f"ProcessedData/AMAX_Events/NIMROD_30mins/all_events_characteristics.csv", index=False)

# Create grouped results, for all events (no duplicates for durations)
### Group by gauge, climate

In [63]:
# grouped_df = grouped_by_gauge_allevents
# group_by_columns = ['Climate', 'gauge_num']
# sampling_duration = 'All'

# group_by_columns_no_climate = [col for col in group_by_columns if col != 'Climate']
# present_df = grouped_df[grouped_df['Climate'] == 'Present'].copy().rename(columns=lambda x: x + '_present' if x not in group_by_columns_no_climate else x)
# future_df = grouped_df[grouped_df['Climate'] == 'Future'].copy().rename(columns=lambda x: x + '_future' if x not in group_by_columns_no_climate else x)

# # Merge present and future data on common columns
# merged_df = pd.merge(present_df, future_df, on=group_by_columns_no_climate, how='outer', suffixes=('_present', '_future'))
# for metric in ['D_mean', 'R', 'D50_mean', 'D50_median', 'D50_P90', 'D50_P10', 'F2_percentage', 'B2_percentage', 'C_percentage', 'F1_percentage', 'B1_percentage']:
#     merged_df[f'{metric}_diff'] = merged_df[f'{metric}_future'] - merged_df[f'{metric}_present']

    
# # Step 1: Define a circular difference function
# def circular_day_difference(present_day, future_day):
#     # Compute the circular difference
#     circular_diff = ((future_day - present_day + 182.625) % 365.25) - 182.625
#     return circular_diff

# # Step 2: Apply the function to the DataFrame columns and create a new column
# merged_df['D_mean_diff_new'] = merged_df.apply(
#     lambda row: circular_day_difference(row['D_mean_future'], row['D_mean_present']), axis=1)
# merged_df[['D_mean_diff', 'D_mean_diff_new']]
# merged_df['dff'] = merged_df['D_mean_diff'] - merged_df['D_mean_diff_new']
# merged_df['D_mean_diff_new'].max()

In [14]:
group_by_columns = ['Climate', 'gauge_num']
grouped_by_gauge_allevents = group_data_calc_means(df_long, 'D50_new', group_by_columns)
grouped_by_gauge_allevents_changes = find_change_values_in_groups_new(grouped_by_gauge_allevents, group_by_columns, 'All')
grouped_by_gauge_allevents_changes.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/grouped_by_gauge_changes_allevents.csv", index=False)

### Group by season, gauge, climate

In [15]:
group_by_columns = ['Climate', 'gauge_num', 'season']
grouped_by_gauge_season_allevents = group_data_calc_means(df_long, 'D50_new', group_by_columns)
grouped_by_gauge_season_allevents_changes = find_change_values_in_groups_new(grouped_by_gauge_season_allevents, group_by_columns, 'All')
grouped_by_gauge_season_allevents_changes.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/grouped_by_gauge_season_changes_allevents.csv", index=False)

# Create grouped results, for all events (for each duration separately)
### Group by gauge, climate and by season, gauge, climate

In [ ]:
each_dur_per_climate_changes = []
each_dur_per_climate_and_season_changes = []

# For each duration in turn
for duration in durations:
    # Get data for just this duration
    this_dur = df_long[df_long['dur_for_which_this_is_amax'].apply(
        lambda x: isinstance(x, list) and str(duration) in x or x == str(duration))]
    
    # Summary of events at each gauge, for this duration, one for present, one for future
    summary_per_climate = group_data_calc_means(this_dur, 'D50_new', ['Climate', 'gauge_num'])
    # Summary of events at each gauge, for this duration, one for each season for present, one for each season for future
    summary_per_climate_and_season = group_data_calc_means(this_dur, 'D50_new', ['Climate', 'gauge_num', 'season'])
    # Reformat, so one row per gauge, with change between present and future in the columns
    summary_per_climate_changes = find_change_values_in_groups_new(summary_per_climate, ['Climate', 'gauge_num'], float(duration))
    # Reformat, so four rows (each season) per gauge, with change between present and future in the columns
    summary_per_climate_season_changes = find_change_values_in_groups_new(summary_per_climate_and_season, ['Climate', 'gauge_num', 'season'], float(duration))
    
    ## Add to lists
    each_dur_per_climate_changes.append(summary_per_climate_changes)
    each_dur_per_climate_and_season_changes.append(summary_per_climate_season_changes)

In [ ]:
# total[['D_mean_diff_new', "D_mean_diff", 'dff']].sort_values(by='dff')[:40]

In [ ]:
total = pd.concat(each_dur_per_climate_changes)
total_season = pd.concat(each_dur_per_climate_and_season_changes)

In [84]:
total.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/grouped_by_gauge_changes_bydur.csv", index=False)
total_season.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/grouped_by_gauge_season_changes_bydur.csv", index=False)

### Save original data
Don't do this higher up, because the json.dumps thing messes up the formatting for later stages of the code

In [40]:
# Create a copy and convert lists to JSON strings before saving - not doing this, messed up formatting a bit
df_long['dur_for_which_this_is_amax'] = df_long['dur_for_which_this_is_amax'].apply(json.dumps)
df_long.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/all_events_characteristics.csv", index=False)
df_long_with_short_durations_kept['dur_for_which_this_is_amax'] = df_long_with_short_durations_kept['dur_for_which_this_is_amax'].apply(json.dumps)
df_long_with_short_durations_kept.to_csv(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/all_events_characteristics_shortdurationskept.csv", index=False)